******************************************************************************************************************************* 
NOTE SUR LE PROJET: Michaël, votre manager, vous incite à sélectionner un ou des kernels Kaggle pour vous faciliter l’analyse exploratoire, la préparation des données et le feature engineering nécessaires à l’élaboration du modèle de scoring. Si vous le faites, vous devez analyser ce ou ces kernels et le ou les adapter pour vous assurer qu’il(s) répond(ent) aux besoins de votre mission. C’est **optionnel**, mais nous vous encourageons à le faire afin de vous permettre de vous focaliser sur l’élaboration du modèle, son optimisation et sa compréhension.                            

Comme ce n'était pas obligatoire, je ne suis pas partie du kernel de Kaggle dans un souhait d'apprentissage. En effet, étant intéressée par ce domaine et mon mentor étant expert dans ce dernier, j'ai eu l'opportunité de réaliser ce projet en immersion dans une agence bancaire.

Même si cela n'était pas demandé, j'ai réalisé une EDA ce qui est de mon point de vue indispensable pour la compréhension des données en notre possession.

Le processing et le feature engineering des tables 'credit_card_balance', 'installments_payments' et 'POS_CASH_balance' seront réalisés dans ce notebook. La jointure permettant d'obtenir l'historique complet sera également réalisée, suivi de l'analyse des fortes corrélations (> 0.8 ou < -0.8). Le traitement des données des jeux d'entraînement et de test, l'EDA et la jointure avec le jeu de données préparé dans ce notebook seront réalisés dans un autre notebook par soucis de temps d'exécution de ce notebook.
*******************************************************************************************************************************

# IMPLEMENTEZ UN MODELE DE SCORING

# Création du modèle de scoring

### Contexte

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser", qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse son crédit, puis classifier la demande en crédit accordé ou refusé. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de transparence vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.

Prêt à dépenser décide donc de développer un dashboard interactif pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement. 

### Missions
- **Mission 1: Construction d'un modèle de scoring donnant une prédiction sur la probabilité de faillite d'un client de façon automatique.**
- **Mission 2: Construction d'un dashboard interactif à destination des gestionnaires de la relation client permettant d'interpréter les prédictions faites par le modèle, et d’améliorer la connaissance client des chargés de relation client.**
- **Mission 3: Mise en production du modèle de scoring de prédiction à l’aide d’une API, ainsi que du dashboard interactif appelant l’API pour les prédictions.**

In [1]:
# Import des librairies classiques pour l'EDA

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mode

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. Le jeu de données 'installments_payments.csv'

## 1.1. Traitement du jeu de données

In [2]:
# Ouverture du jeu de données
file_1 = pd.read_csv("installments_payments.csv", sep=",")
pd.set_option("Display.max_rows", None)
pd.set_option("Display.max_columns", None)
file_1.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [3]:
# Copie du jeu de données
installments = file_1.copy()

In [4]:
# Informations sur le jeu de données
installments.info(verbose=True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13605401 entries, 0 to 13605400
Data columns (total 8 columns):
 #   Column                  Non-Null Count     Dtype  
---  ------                  --------------     -----  
 0   SK_ID_PREV              13605401 non-null  int64  
 1   SK_ID_CURR              13605401 non-null  int64  
 2   NUM_INSTALMENT_VERSION  13605401 non-null  float64
 3   NUM_INSTALMENT_NUMBER   13605401 non-null  int64  
 4   DAYS_INSTALMENT         13605401 non-null  float64
 5   DAYS_ENTRY_PAYMENT      13602496 non-null  float64
 6   AMT_INSTALMENT          13605401 non-null  float64
 7   AMT_PAYMENT             13602496 non-null  float64
dtypes: float64(5), int64(3)
memory usage: 830.4 MB


**Dans ce jeu de données, seules les variables 'AMT_PAYMENT' et 'AMT_INSTALMENT' seront retenues même si nous sommes en présence de données manquantes pour la variable 'AMT_PAYMENT'.**

*L'agrégation pour la variable 'AMT_PAYMENT', variable renseignant le montant réellement versé, sera réalisée sur la **MOYENNE** (mean).*

*La variable AMT_INSTALMENT renseigne sur le montant que le client devrait payer. L'agrégation sera aussi réalisée sur la **MOYENNE** (mean).*

*A NOTER: Au départ, je souhaitais créer une variable 'PAYMENT_PERC' renseignant sur le pourcentage réellement payé par rapport au montant devant être payé et réalisér une agrégation sur la **MOYENNE** (mean). Comme dans le précédent notebook, des valeurs de 0 sont présentes au niveau de la variable 'AMT_INSTALMENT', conduisant donc à des valeurs infinies. De plus, la variable 'AMT_PAYMENT' présente des valeurs manquantes, rendant ainsi la tâche davantage complexe. Enfin, pour l'avoir testé, des valeurs incohéhérentes apppaissaient.*

*CONCLUSION: Aucun feature engineering ne sera réalisé et les valeurs manquantes concernant la variable 'AMT_PAYMENT' seront imputées par la valeur -2.*

**1. PREUVE DE L'EXISTENCE DE LA VALEUR ZERO DANS LA VARIABLE 'AMT_INSTALMENT'**

In [5]:
# Création du dataframe contenant les valeurs 0
zero_amt_instalment = installments[installments["AMT_INSTALMENT"] == 0]
zero_amt_instalment.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
43318,1822101,170117,1.0,5,-2085.0,-2095.0,0.0,11431.62
71362,1822101,170117,1.0,10,-1935.0,-1940.0,0.0,11431.62
516837,1822101,170117,1.0,3,-2145.0,-2155.0,0.0,11431.62
561460,1822101,170117,1.0,4,-2115.0,-2127.0,0.0,11431.62
1341687,1171606,106652,1.0,8,-2922.0,-467.0,0.0,6.30


In [6]:
# Nombre de lignes concernées
zero_amt_instalment.shape[0]

290

**2. IMPUTATION PAR -2 POUR LES VALEURS MANQUANTES DE LA VARIABLE 'AMT_PAYMENT'**

In [7]:
# Imputation par la valeur -2
installments['AMT_PAYMENT'] = installments['AMT_PAYMENT'].fillna(value=-2)

In [8]:
# Description de la variable pour vérifier la bonne imputation
installments['AMT_PAYMENT'].describe()

count    1.360540e+07
mean     1.723454e+04
std      5.473052e+04
min     -2.000000e+00
25%      3.394170e+03
50%      8.122185e+03
75%      1.610591e+04
max      3.771488e+06
Name: AMT_PAYMENT, dtype: float64

**3. AGREGATION PAR LA MOYENNE POUR LES 2 VARIABLES**

In [9]:
# Agrégration des 2 variables par la moyenne
installments_payment = installments.groupby("SK_ID_CURR").agg({"AMT_PAYMENT":"mean", "AMT_INSTALMENT":"mean"}).reset_index()

In [10]:
# Description du jeu de données
installments_payment.describe()

,SK_ID_CURR,AMT_PAYMENT,AMT_INSTALMENT
count,339587.000000,3.395870e+05,3.395870e+05
mean,278154.892278,1.900093e+04,1.861577e+04
std,102880.492598,2.523025e+04,2.346509e+04
min,100001.000000,-2.000000e+00,0.000000e+00
25%,189042.500000,7.580438e+03,7.897470e+03
50%,278238.000000,1.240217e+04,1.272616e+04
75%,367315.500000,2.164076e+04,2.164516e+04
max,456255.000000,2.504590e+06,2.504590e+06


**A NOTER:** Afin d'éviter des valeurs intermédiaires négatives, une ré-imputation par la valeur -2 sera réalisée pour la variable 'AMT_PAYMENT'.

In [11]:
# Création du jeu de données pour la ré-imputation par la valeur -2 pour la variable 'AMT_PAYMENT'
negative_amt_payment = installments_payment[installments_payment["AMT_PAYMENT"] < -0.00001]
negative_amt_payment.head()

,SK_ID_CURR,AMT_PAYMENT,AMT_INSTALMENT
49589,151950,-2.0,312588.000
51163,153598,-2.0,233338.680
114447,220039,-2.0,54180.090
124175,230272,-2.0,79425.045
155797,263534,-2.0,99838.530


In [12]:
# Ré-imputation par la valeur -2 pour la variable 'AMT_PAYMENT'
negative_amt_payment = negative_amt_payment.copy()

for idx in negative_amt_payment.index:
    installments_payment.loc[idx, "AMT_PAYMENT"] = -2

In [13]:
# Informations sur le jeu de données
installments_payment.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 339587 entries, 0 to 339586
Data columns (total 3 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   SK_ID_CURR      339587 non-null  int64  
 1   AMT_PAYMENT     339587 non-null  float64
 2   AMT_INSTALMENT  339587 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 7.8 MB


**Nous sommes en présence d'un jeu de données complet et entièrement numérique.**

## 1.2. Jointure avec le jeu de données 'aggregated_previous_application'

**RAPPEL:** Ce jeu de données est issu du notebook 'Ple_Coline_2_notebook_previous_application_092023'

### 1. Ouverture et copie du jeu de données

In [14]:
# Ouverture du jeu de données
file_2 = pd.read_csv("aggregated_previous_application.csv", sep=",")
pd.set_option("Display.max_rows", None)
pd.set_option("Display.max_columns", None)
file_2.head()

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1,0.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1,-1.0


In [15]:
# Copie du jeu de données
aggregated_previous_application = file_2.copy()

### 2. La jointure en elle-même

In [16]:
# Jointure avec le jeu de données 'aggregated_previous_application'
merge_previous_1 = pd.merge(aggregated_previous_application, installments_payment, on="SK_ID_CURR", how='left')
merge_previous_1.head()

,SK_ID_CURR,AMT_ANNUITY_mean,AMT_CREDIT_mean,AMT_CREDIT_sum,AMT_APPLICATION_mean,AMT_GOODS_PRICE_mean,AMT_DOWN_PAYMENT_mean,DAYS_DECISION_mean,CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,AMT_PAYMENT,AMT_INSTALMENT
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,5885.132143,5885.132143
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1,0.0,11559.247105,11559.247105
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1.0,64754.586000,64754.586000
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,7096.155000,7096.155000
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1,-1.0,6240.205000,6240.205000


In [17]:
# Vérification du nombre de clients (attendu: 338857)
print(f"Nous sommes toujours bien en présence de {merge_previous_1.shape[0]} clients.")

Nous sommes toujours bien en présence de 338857 clients.


# 2. Le jeu de données 'credit_card_balance.csv'

## 2.1. Ouverture et informations générales sur le jeu de données

In [18]:
# Ouverture du jeu de données
file_3 = pd.read_csv("credit_card_balance.csv", sep=",")
pd.set_option("Display.max_rows", None)
pd.set_option("Display.max_columns", None)
file_3.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [19]:
# Copie du jeu de données
credit_cards = file_3.copy()

In [20]:
# Informations sur le jeu de données
credit_cards.info(verbose=True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3840312 entries, 0 to 3840311
Data columns (total 23 columns):
 #   Column                      Non-Null Count    Dtype  
---  ------                      --------------    -----  
 0   SK_ID_PREV                  3840312 non-null  int64  
 1   SK_ID_CURR                  3840312 non-null  int64  
 2   MONTHS_BALANCE              3840312 non-null  int64  
 3   AMT_BALANCE                 3840312 non-null  float64
 4   AMT_CREDIT_LIMIT_ACTUAL     3840312 non-null  int64  
 5   AMT_DRAWINGS_ATM_CURRENT    3090496 non-null  float64
 6   AMT_DRAWINGS_CURRENT        3840312 non-null  float64
 7   AMT_DRAWINGS_OTHER_CURRENT  3090496 non-null  float64
 8   AMT_DRAWINGS_POS_CURRENT    3090496 non-null  float64
 9   AMT_INST_MIN_REGULARITY     3535076 non-null  float64
 10  AMT_PAYMENT_CURRENT         3072324 non-null  float64
 11  AMT_PAYMENT_TOTAL_CURRENT   3840312 non-null  float64
 12  AMT_RECEIVABLE_PRINCIPAL    3840312 non-null  float64
 1

**Concernant ce jeu de données, étant déjà en possession d'un nombre important de variables, une seule variable 'générale' sera conservée car elle est amplement suffisante pour notre projet et n'est pas redondante:**
- **AMT_CREDIT_LIMIT_ACTUAL**: Cette information peut donner aux prêteurs un aperçu de la confiance sur la solvabilité du client à ce moment-là. Une limite plus élevée peut indiquer une plus grande confiance dans la capacité de remboursement du client, tandis qu'une limite plus faible peut indiquer le contraire.

**A NOTER: Afin d'éviter le maximum de redondance, la variable 'SK_DPD' sera prise dans le jeu de données 'POS_CASH_balance.csv'.**

## 2.2. Agrégations des données

*Pour cette variable, une aggrégation sur la **MOYENNE** (mean) sera réalisée.*

In [21]:
aggregated_credit_cards = credit_cards.groupby("SK_ID_CURR").agg({"AMT_CREDIT_LIMIT_ACTUAL":"mean"})

aggregated_credit_cards.head()

,AMT_CREDIT_LIMIT_ACTUAL
SK_ID_CURR,
100006,270000.000000
100011,164189.189189
100013,131718.750000
100021,675000.000000
100023,135000.000000


**Nous remarquons que tous les clients n'apparaissent pas. Ceci est peut-être du au fait que certains clients ne possèdent pas de carte bancaire. Par conséquent, cette variable ne sera pas prise en compte.**

# 3. Le jeu de données 'POS_CASH_balance.csv'

In [22]:
# Ouverture du jeu de données
file_4 = pd.read_csv("POS_CASH_balance.csv", sep=",")
pd.set_option("Display.max_rows", None)
pd.set_option("Display.max_columns", None)
file_4.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [23]:
# Copie du jeu de données
pos_cash = file_4.copy()

In [24]:
# Informations sur le jeu de données
pos_cash.info(verbose=True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10001358 entries, 0 to 10001357
Data columns (total 8 columns):
 #   Column                 Non-Null Count     Dtype  
---  ------                 --------------     -----  
 0   SK_ID_PREV             10001358 non-null  int64  
 1   SK_ID_CURR             10001358 non-null  int64  
 2   MONTHS_BALANCE         10001358 non-null  int64  
 3   CNT_INSTALMENT         9975287 non-null   float64
 4   CNT_INSTALMENT_FUTURE  9975271 non-null   float64
 5   NAME_CONTRACT_STATUS   10001358 non-null  object 
 6   SK_DPD                 10001358 non-null  int64  
 7   SK_DPD_DEF             10001358 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 610.4+ MB


*Pour ce jeu de données, nous n'utiliserons que la variable 'SK_DPD' qui sera agrégée sur le **NOMBRE** (count). Cette dernière ne présente pas de donnée manquante.*

In [25]:
# Vérification de l'absence d'anomalie
pos_cash["SK_DPD"].describe()

count    1.000136e+07
mean     1.160693e+01
std      1.327140e+02
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      4.231000e+03
Name: SK_DPD, dtype: float64

*Cette variable ne semble pas présenter d'anomalie.*

**1. AGREGATION DE LA VARIABLE**

In [26]:
aggregated_pos_cash = pos_cash.groupby("SK_ID_CURR").agg({"SK_DPD":"count"})

# Renommage de la variable pour plus de clarté
aggregated_pos_cash= aggregated_pos_cash.rename(columns={"SK_DPD":"prev_SK_DPD_count"})

aggregated_pos_cash.head()

,prev_SK_DPD_count
SK_ID_CURR,
100001,9
100002,19
100003,28
100004,4
100005,11


**2. JOINTURE AVEC LE DATAFRAME PRECEDENT**

In [27]:
# Création du dataframe
previous_aggregated = pd.merge(merge_previous_1, aggregated_pos_cash, on='SK_ID_CURR', how='left')

# Renommage pour plus de clarté SK_DPD pour plus de clarté
previous_aggregated = previous_aggregated.rename(columns={"AMT_ANNUITY_mean":"prev_AMT_ANNUITY_mean", "AMT_CREDIT_mean":"prev_AMT_CREDIT_mean",
                                                          "AMT_CREDIT_sum":"prev_AMT_CREDIT_sum", "AMT_APPLICATION_mean":"prev_AMT_APPLICATION_mean",
                                                          "AMT_GOODS_PRICE_mean":"prev_AMT_GOODS_PRICE_mean", "AMT_DOWN_PAYMENT_mean":"prev_AMT_DOWN_PAYMENT_mean",
                                                         "DAYS_DECISION_mean":"prev_DAYS_DECISION_mean", "CNT_PAYMENT_mean":"prev_CNT_PAYMENT_mean", 
                                                          "AMT_INSTALMENT": "prev_AMT_INSTALMENT_mean","AMT_PAYMENT":"prev_AMT_PAYMENT_mean"})

previous_aggregated.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1,0.0,5885.132143,5885.132143,9.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1,0.0,11559.247105,11559.247105,19.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1,1.0,64754.586000,64754.586000,28.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,7096.155000,7096.155000,4.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1,-1.0,6240.205000,6240.205000,11.0


In [28]:
# Description du jeu de données
previous_aggregated.describe()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count
count,338857.000000,338857.000000,3.388570e+05,3.388570e+05,3.388570e+05,3.388570e+05,3.388570e+05,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,338857.000000,3.377950e+05,3.377950e+05,335852.000000
mean,278149.909581,10151.158098,1.297287e+05,6.197732e+05,1.178547e+05,1.178546e+05,3.998877e+03,919.288946,11.680744,3.059642,0.933488,0.857819,0.078015,2.207123,2.151796,0.570046,-0.470552,-0.280334,1.907179e+04,1.868335e+04,29.717709
std,102879.193103,8102.821542,1.328592e+05,7.667626e+05,1.230787e+05,1.230787e+05,1.140887e+04,574.658977,7.216252,2.135404,1.723006,1.830574,0.327991,3.298588,1.820743,1.024132,0.743967,0.636467,2.527592e+04,2.350620e+04,24.512364
min,100001.000000,-2.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+00,-2.000000e+00,2.000000,-2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,-2.000000e+00,0.000000e+00,1.000000
25%,189061.000000,5054.247000,4.962420e+04,1.307035e+05,4.578300e+04,4.578300e+04,0.000000e+00,471.500000,6.615385,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-1.000000,-1.000000,7.630205e+03,7.949511e+03,12.000000
50%,278221.000000,8124.813750,8.965800e+04,3.225600e+05,8.157900e+04,8.157900e+04,1.242750e+03,788.250000,10.333333,3.000000,0.000000,0.000000,0.000000,1.000000,2.000000,0.000000,-1.000000,0.000000,1.245588e+04,1.277967e+04,22.000000
75%,367302.000000,12794.130000,1.617705e+05,8.113725e+05,1.443690e+05,1.443690e+05,4.338000e+03,1240.666667,14.666667,4.000000,1.000000,1.000000,0.000000,3.000000,3.000000,1.000000,0.000000,0.000000,2.172342e+04,2.171892e+04,39.000000
max,456255.000000,290358.000000,4.050000e+06,1.048602e+07,4.050000e+06,4.050000e+06,1.012499e+06,2922.000000,72.000000,27.000000,58.000000,68.000000,21.000000,66.000000,48.000000,31.000000,1.000000,1.000000,2.504590e+06,2.504590e+06,295.000000


In [29]:
# Informations sur le jeu de données
previous_aggregated.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   SK_ID_CURR                      338857 non-null  int64  
 1   prev_AMT_ANNUITY_mean           338857 non-null  float64
 2   prev_AMT_CREDIT_mean            338857 non-null  float64
 3   prev_AMT_CREDIT_sum             338857 non-null  float64
 4   prev_AMT_APPLICATION_mean       338857 non-null  float64
 5   prev_AMT_GOODS_PRICE_mean       338857 non-null  float64
 6   prev_AMT_DOWN_PAYMENT_mean      338857 non-null  float64
 7   prev_DAYS_DECISION_mean         338857 non-null  float64
 8   prev_CNT_PAYMENT_mean           338857 non-null  float64
 9   prev_approved                   338857 non-null  float64
 10  prev_canceled                   338857 non-null  float64
 11  prev_refused                    338857 non-null  float64
 12  prev_unused_offe

**A NOTER:** Nous sommes en présence de données manquantes pour les 3 nouvelles variables aggrégées. Ceci ne me semble pas anormal car lorsque nous effectuons des moyennes avec un numérateur de 0, Pandas revoie un NaN. Plour les clients ne possédédant pas d'historique de prêt accepté, les 3 variables seront imputées par la valeur 0.

**3. IMPUTATION DES VALEURS MANQUANTES**

In [30]:
# Création du dataframe comprenant les valeurs manquantes uniquement pour les variables 'prev_AMT_PAYMENT_mean' et 'prev_AMT_INSTALMENT_mean'
nan_installments = previous_aggregated[(previous_aggregated["prev_AMT_PAYMENT_mean"].isna()) | (previous_aggregated["prev_AMT_INSTALMENT_mean"].isna())]
nan_installments.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count
113,100120,0.000,0.0,0.0,0.0,0.0,0.0,17.333333,-2.000000,0.0,3.0,0.0,0.0,3.0,0.0,0.0,-1,-1.0,NaN,NaN,19.0
150,100160,17093.790,123733.5,371200.5,123733.5,123733.5,0.0,40.000000,6.000000,2.0,0.0,0.0,1.0,0.0,3.0,0.0,-1,-1.0,NaN,NaN,NaN
262,100278,5218.230,44346.0,133038.0,43485.0,43485.0,4500.0,234.666667,15.333333,1.0,0.0,2.0,0.0,0.0,3.0,0.0,0,-1.0,NaN,NaN,27.0
444,100471,14607.135,202455.0,202455.0,202455.0,202455.0,0.0,24.000000,18.000000,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,NaN,NaN,NaN
582,100614,0.000,0.0,0.0,0.0,0.0,0.0,286.000000,-2.000000,0.0,1.0,0.0,1.0,0.0,1.0,1.0,-1,-1.0,NaN,NaN,11.0


**A NOTER: Présence d'incohérences car des clients n'ayant jamais eu de prêt accepté possèdent un nombre pour la variable 'prev_SK_DPD_count'!!!**
    
**2 STRATEGIES D'IMPUTATION**
- **Mise à zéro des 3 variables pour les clients n'ayant jamais eu de crédit accordé.**
- **Imputation par -2 pour le reste des variables manquantes.**

In [31]:
# Création du dataframe
no_prev_history = previous_aggregated[previous_aggregated["prev_approved"] == 0]
no_prev_history.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count
113,100120,0.0,0.0,0.0,0.0,0.0,0.0,17.333333,-2.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,-1,-1.0,NaN,NaN,19.0
475,100505,0.0,0.0,0.0,0.0,0.0,0.0,333.000000,36.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,-1,-1.0,6173.031557,7196.983653,NaN
582,100614,0.0,0.0,0.0,0.0,0.0,0.0,286.000000,-2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,-1,-1.0,NaN,NaN,11.0
991,101040,0.0,0.0,0.0,0.0,0.0,0.0,36.333333,13.0,0.0,3.0,3.0,0.0,6.0,0.0,0.0,-1,-1.0,NaN,NaN,NaN
1234,101291,0.0,0.0,0.0,0.0,0.0,0.0,2169.333333,10.0,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0,-1.0,NaN,NaN,NaN


In [32]:
# Imputation par 0 pour les clients sans crédit accepté
no_prev_history= no_prev_history.copy()

for idx in no_prev_history.index:
    previous_aggregated.loc[idx, "prev_SK_DPD_count"] = 0
    previous_aggregated.loc[idx, "prev_AMT_PAYMENT_mean"] = 0
    previous_aggregated.loc[idx, "prev_AMT_INSTALMENT_mean"] = 0

In [33]:
# Informations sur le jeu de données
previous_aggregated.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   SK_ID_CURR                      338857 non-null  int64  
 1   prev_AMT_ANNUITY_mean           338857 non-null  float64
 2   prev_AMT_CREDIT_mean            338857 non-null  float64
 3   prev_AMT_CREDIT_sum             338857 non-null  float64
 4   prev_AMT_APPLICATION_mean       338857 non-null  float64
 5   prev_AMT_GOODS_PRICE_mean       338857 non-null  float64
 6   prev_AMT_DOWN_PAYMENT_mean      338857 non-null  float64
 7   prev_DAYS_DECISION_mean         338857 non-null  float64
 8   prev_CNT_PAYMENT_mean           338857 non-null  float64
 9   prev_approved                   338857 non-null  float64
 10  prev_canceled                   338857 non-null  float64
 11  prev_refused                    338857 non-null  float64
 12  prev_unused_offe

In [34]:
# Imputation par la valeur -2 pour le reste des valeurs manquantes
previous_aggregated = previous_aggregated.fillna(value=-2)

In [35]:
# Informations sur le jeu de données final
previous_aggregated.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 338857 entries, 0 to 338856
Data columns (total 21 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   SK_ID_CURR                      338857 non-null  int64  
 1   prev_AMT_ANNUITY_mean           338857 non-null  float64
 2   prev_AMT_CREDIT_mean            338857 non-null  float64
 3   prev_AMT_CREDIT_sum             338857 non-null  float64
 4   prev_AMT_APPLICATION_mean       338857 non-null  float64
 5   prev_AMT_GOODS_PRICE_mean       338857 non-null  float64
 6   prev_AMT_DOWN_PAYMENT_mean      338857 non-null  float64
 7   prev_DAYS_DECISION_mean         338857 non-null  float64
 8   prev_CNT_PAYMENT_mean           338857 non-null  float64
 9   prev_approved                   338857 non-null  float64
 10  prev_canceled                   338857 non-null  float64
 11  prev_refused                    338857 non-null  float64
 12  prev_unused_offe

**Tout est OK**                                                                             

# 5. Jointure avec le jeu de données home_credit

**ATTENTION**: Nous avons moins de clients dans le Bureau de crédits que dans previous_application. Par conséquent, la jointure doit se faire avec **how="outer"** pour conserver tous les clients. Cette dernière engendrera l'apparition de valeurs manquantes qui ne seront pas traitées avant l'agrégation avec les jeux de données d'entrainement et de test.

In [36]:
# Ouverture du jeu de données 'home_credit' issu du 1er notebook
file_5= pd.read_csv("home_credit.csv", sep=",")
pd.set_option("Display.max_rows", None)
pd.set_option("Display.max_columns", None)
file_5.head()

,SK_ID_CURR,home_total_loans,home_active_sum,home_closed_sum,home_bad_debt_sum,home_sold_sum,home_DAYS_CREDIT_min,home_CREDIT_DAY_OVERDUE_mean,home_DAYS_CREDIT_ENDDATE_mean,home_DAYS_ENDDATE_FACT_min,home_CNT_CREDIT_PROLONG_sum,home_AMT_CREDIT_SUM_sum,home_AMT_CREDIT_SUM_mean,home_AMT_CREDIT_SUM_DEBT_sum,home_AMT_CREDIT_SUM_LIMIT_mean,home_AMT_CREDIT_SUM_OVERDUE_mean,home_AMT_CREDIT_MAX_OVERDUE_mean,home_AMT_ANNUITY_mean
0,100001,7,3.0,4.0,0.0,0.0,49,0.0,441.571429,-2.0,0,1453365.000,207623.571429,596686.5,-2.000000,0.0,-2.000000,3545.357143
1,100002,8,2.0,6.0,0.0,0.0,103,0.0,115.625000,-2.0,0,865055.565,108131.945625,245775.0,3997.570625,0.0,1049.893125,0.000000
2,100003,4,1.0,3.0,0.0,0.0,606,0.0,304.000000,-2.0,0,1017400.500,254350.125000,0.0,202500.000000,0.0,0.000000,-2.000000
3,100004,2,0.0,2.0,0.0,0.0,408,0.0,0.000000,382.0,0,189037.800,94518.900000,0.0,0.000000,0.0,-2.000000,0.000000
4,100005,3,2.0,1.0,0.0,0.0,62,0.0,482.000000,-2.0,0,657126.000,219042.000000,568408.5,0.000000,0.0,-2.000000,1420.500000


In [37]:
# Copie du fichier
credit_home = file_5.copy()

In [38]:
# Création du dataframe
final_merge = pd.merge(previous_aggregated, credit_home, on='SK_ID_CURR', how='outer')
final_merge.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count,home_total_loans,home_active_sum,home_closed_sum,home_bad_debt_sum,home_sold_sum,home_DAYS_CREDIT_min,home_CREDIT_DAY_OVERDUE_mean,home_DAYS_CREDIT_ENDDATE_mean,home_DAYS_ENDDATE_FACT_min,home_CNT_CREDIT_PROLONG_sum,home_AMT_CREDIT_SUM_sum,home_AMT_CREDIT_SUM_mean,home_AMT_CREDIT_SUM_DEBT_sum,home_AMT_CREDIT_SUM_LIMIT_mean,home_AMT_CREDIT_SUM_OVERDUE_mean,home_AMT_CREDIT_MAX_OVERDUE_mean,home_AMT_ANNUITY_mean
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5885.132143,5885.132143,9.0,7.0,3.0,4.0,0.0,0.0,49.0,0.0,441.571429,-2.0,0.0,1453365.000,207623.571429,596686.5,-2.000000,0.0,-2.000000,3545.357143
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,11559.247105,11559.247105,19.0,8.0,2.0,6.0,0.0,0.0,103.0,0.0,115.625000,-2.0,0.0,865055.565,108131.945625,245775.0,3997.570625,0.0,1049.893125,0.000000
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,64754.586000,64754.586000,28.0,4.0,1.0,3.0,0.0,0.0,606.0,0.0,304.000000,-2.0,0.0,1017400.500,254350.125000,0.0,202500.000000,0.0,0.000000,-2.000000
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7096.155000,7096.155000,4.0,2.0,0.0,2.0,0.0,0.0,408.0,0.0,0.000000,382.0,0.0,189037.800,94518.900000,0.0,0.000000,0.0,-2.000000,0.000000
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1.0,-1.0,6240.205000,6240.205000,11.0,3.0,2.0,1.0,0.0,0.0,62.0,0.0,482.000000,-2.0,0.0,657126.000,219042.000000,568408.5,0.000000,0.0,-2.000000,1420.500000


In [39]:
# Nombre de clients
print(f"{final_merge.shape[0]} clients possèdent un historique de prêt.")

353577 clients possèdent un historique de prêt.


In [40]:
# Informations sur le jeu de données
final_merge.info(verbose=True, show_counts= True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353577 entries, 0 to 353576
Data columns (total 38 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        353577 non-null  int64  
 1   prev_AMT_ANNUITY_mean             338857 non-null  float64
 2   prev_AMT_CREDIT_mean              338857 non-null  float64
 3   prev_AMT_CREDIT_sum               338857 non-null  float64
 4   prev_AMT_APPLICATION_mean         338857 non-null  float64
 5   prev_AMT_GOODS_PRICE_mean         338857 non-null  float64
 6   prev_AMT_DOWN_PAYMENT_mean        338857 non-null  float64
 7   prev_DAYS_DECISION_mean           338857 non-null  float64
 8   prev_CNT_PAYMENT_mean             338857 non-null  float64
 9   prev_approved                     338857 non-null  float64
 10  prev_canceled                     338857 non-null  float64
 11  prev_refused                      338857 non-null  f

In [41]:
# Vérification de l'absence de doublon sur la variable 'SK_ID_CURR'
final_doublons = final_merge["SK_ID_CURR"].duplicated().sum()
print(f"En se basant sur l\'identifiant du client, nous sommes en présence de {final_doublons} doublon(s).")

En se basant sur l'identifiant du client, nous sommes en présence de 0 doublon(s).


# 6. Données manquantes et feature engineering

**Concernant les données manquantes, nous sommes face à 2 cas de figure:**
- **Les variables numériques comprenant de 'vraies' valeurs numériques**: Dans ce cas de figure, l'imputation peut se faire par 0 car il n'y a vraiment pas d'historique dans le Bureau de crédits et/ou dans previous application.
- **Les 2 variables catégorielles encodées n'étant pas numériques ("prev_NAME_TYPE_SUITE", "prev_NFLAG_INSURED_ON_APPROVAL"**: Imputation par -1 (comme pour les valeurs inconnues).
    
**Stratégie**: N'ayant que 2 variables catégorielles encodées, ces dernières seront imputées avant pour plus de rapidité.

## 6.1. Imputation des valeurs manquantes

In [42]:
columns_to_fill_cat = ["prev_NAME_TYPE_SUITE", "prev_NFLAG_INSURED_ON_APPROVAL"]  

# Imputation des valeurs manquantes par -1 pour ces colonnes
for col in columns_to_fill_cat:
    final_merge[col] = final_merge[col].fillna(value=-1)

# Imputation par 0 pour les vraies variables numériques
final_merge = final_merge.fillna(value=0)

In [43]:
# Informations sur le jeu de données
final_merge.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353577 entries, 0 to 353576
Data columns (total 38 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        353577 non-null  int64  
 1   prev_AMT_ANNUITY_mean             353577 non-null  float64
 2   prev_AMT_CREDIT_mean              353577 non-null  float64
 3   prev_AMT_CREDIT_sum               353577 non-null  float64
 4   prev_AMT_APPLICATION_mean         353577 non-null  float64
 5   prev_AMT_GOODS_PRICE_mean         353577 non-null  float64
 6   prev_AMT_DOWN_PAYMENT_mean        353577 non-null  float64
 7   prev_DAYS_DECISION_mean           353577 non-null  float64
 8   prev_CNT_PAYMENT_mean             353577 non-null  float64
 9   prev_approved                     353577 non-null  float64
 10  prev_canceled                     353577 non-null  float64
 11  prev_refused                      353577 non-null  f

**Nous sommes bien en présence d'un jeu de données complet, ce qui facilitera le feature engineering.**

## 6.2. Feature engineering

**Ayant un jeu de données obtenu à partir de 2 jeux de données construits différemment, afin de ne pas introduire de biais, le feature engineering sera très léger.** 

### 1. Le pourcentage des différents types de prêts

**A NOTER**: Dans les jeux de données d'entraînement et de test, les types de crédits sont référencés en revolving_loans et cash_loans, ces derniers étant largement majoritaires (travail réalisé en parallèle).

Le pourcentage de revolving loans est très similaire à celui présent dans les jeux d'entraînement et de test. Par conséquent, nous pourrions envisager de regrouper les 'Consumer loans' et 'Cash loans'. Néanmoins, ayant peur d'introduire un biais en faisant cela, les 3 types de crédits seront conservés en l'état mais seront exprimés en pourcentages...

In [44]:
# Total de différents types de prêts
final_merge["prev_type_loans"] = final_merge["prev_cash_loans"] + final_merge["prev_consumer_loans"] + final_merge["prev_revolving_loans"]

# Pourcentage des cash_loans et des revolving_loans
final_merge["prev_cash_loans_perc"] = round((final_merge["prev_cash_loans"]*100)/final_merge["prev_type_loans"],1)
final_merge["prev_revolving_loans_perc"] = round((final_merge["prev_revolving_loans"]*100)/final_merge["prev_type_loans"],1)

# Vérification de la présence des nouvelles variables
final_merge.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_cash_loans,prev_consumer_loans,prev_revolving_loans,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count,home_total_loans,home_active_sum,home_closed_sum,home_bad_debt_sum,home_sold_sum,home_DAYS_CREDIT_min,home_CREDIT_DAY_OVERDUE_mean,home_DAYS_CREDIT_ENDDATE_mean,home_DAYS_ENDDATE_FACT_min,home_CNT_CREDIT_PROLONG_sum,home_AMT_CREDIT_SUM_sum,home_AMT_CREDIT_SUM_mean,home_AMT_CREDIT_SUM_DEBT_sum,home_AMT_CREDIT_SUM_LIMIT_mean,home_AMT_CREDIT_SUM_OVERDUE_mean,home_AMT_CREDIT_MAX_OVERDUE_mean,home_AMT_ANNUITY_mean,prev_type_loans,prev_cash_loans_perc,prev_revolving_loans_perc
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5885.132143,5885.132143,9.0,7.0,3.0,4.0,0.0,0.0,49.0,0.0,441.571429,-2.0,0.0,1453365.000,207623.571429,596686.5,-2.000000,0.0,-2.000000,3545.357143,1.0,0.0,0.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,-1.0,0.0,11559.247105,11559.247105,19.0,8.0,2.0,6.0,0.0,0.0,103.0,0.0,115.625000,-2.0,0.0,865055.565,108131.945625,245775.0,3997.570625,0.0,1049.893125,0.000000,1.0,0.0,0.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,2.0,0.0,1.0,1.0,64754.586000,64754.586000,28.0,4.0,1.0,3.0,0.0,0.0,606.0,0.0,304.000000,-2.0,0.0,1017400.500,254350.125000,0.0,202500.000000,0.0,0.000000,-2.000000,3.0,33.3,0.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,7096.155000,7096.155000,4.0,2.0,0.0,2.0,0.0,0.0,408.0,0.0,0.000000,382.0,0.0,189037.800,94518.900000,0.0,0.000000,0.0,-2.000000,0.000000,1.0,0.0,0.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,-1.0,-1.0,6240.205000,6240.205000,11.0,3.0,2.0,1.0,0.0,0.0,62.0,0.0,482.000000,-2.0,0.0,657126.000,219042.000000,568408.5,0.000000,0.0,-2.000000,1420.500000,2.0,50.0,0.0


In [45]:
# Eliminations des 3 variables initiales devenues inutiles
final_merge = final_merge.drop(["prev_cash_loans", "prev_consumer_loans", "prev_revolving_loans"], axis = 1)
final_merge.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_approved,prev_canceled,prev_refused,prev_unused_offer,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count,home_total_loans,home_active_sum,home_closed_sum,home_bad_debt_sum,home_sold_sum,home_DAYS_CREDIT_min,home_CREDIT_DAY_OVERDUE_mean,home_DAYS_CREDIT_ENDDATE_mean,home_DAYS_ENDDATE_FACT_min,home_CNT_CREDIT_PROLONG_sum,home_AMT_CREDIT_SUM_sum,home_AMT_CREDIT_SUM_mean,home_AMT_CREDIT_SUM_DEBT_sum,home_AMT_CREDIT_SUM_LIMIT_mean,home_AMT_CREDIT_SUM_OVERDUE_mean,home_AMT_CREDIT_MAX_OVERDUE_mean,home_AMT_ANNUITY_mean,prev_type_loans,prev_cash_loans_perc,prev_revolving_loans_perc
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,1.0,0.0,0.0,0.0,1.0,0.0,5885.132143,5885.132143,9.0,7.0,3.0,4.0,0.0,0.0,49.0,0.0,441.571429,-2.0,0.0,1453365.000,207623.571429,596686.5,-2.000000,0.0,-2.000000,3545.357143,1.0,0.0,0.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,1.0,0.0,0.0,0.0,-1.0,0.0,11559.247105,11559.247105,19.0,8.0,2.0,6.0,0.0,0.0,103.0,0.0,115.625000,-2.0,0.0,865055.565,108131.945625,245775.0,3997.570625,0.0,1049.893125,0.000000,1.0,0.0,0.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,3.0,0.0,0.0,0.0,1.0,1.0,64754.586000,64754.586000,28.0,4.0,1.0,3.0,0.0,0.0,606.0,0.0,304.000000,-2.0,0.0,1017400.500,254350.125000,0.0,202500.000000,0.0,0.000000,-2.000000,3.0,33.3,0.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,1.0,0.0,0.0,0.0,0.0,0.0,7096.155000,7096.155000,4.0,2.0,0.0,2.0,0.0,0.0,408.0,0.0,0.000000,382.0,0.0,189037.800,94518.900000,0.0,0.000000,0.0,-2.000000,0.000000,1.0,0.0,0.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,1.0,0.0,0.0,-1.0,-1.0,6240.205000,6240.205000,11.0,3.0,2.0,1.0,0.0,0.0,62.0,0.0,482.000000,-2.0,0.0,657126.000,219042.000000,568408.5,0.000000,0.0,-2.000000,1420.500000,2.0,50.0,0.0


### 2. Le nombre total de prêts acceptés

In [46]:
final_merge["total_accepted_loans"] = final_merge["home_total_loans"] + final_merge["prev_approved"]

In [47]:
# Elimination des 2 variables d'origine devenues inutiles
final_merge = final_merge.drop(["home_total_loans", "prev_approved"], axis=1)
final_merge.head()

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_APPLICATION_mean,prev_AMT_GOODS_PRICE_mean,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_canceled,prev_refused,prev_unused_offer,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count,home_active_sum,home_closed_sum,home_bad_debt_sum,home_sold_sum,home_DAYS_CREDIT_min,home_CREDIT_DAY_OVERDUE_mean,home_DAYS_CREDIT_ENDDATE_mean,home_DAYS_ENDDATE_FACT_min,home_CNT_CREDIT_PROLONG_sum,home_AMT_CREDIT_SUM_sum,home_AMT_CREDIT_SUM_mean,home_AMT_CREDIT_SUM_DEBT_sum,home_AMT_CREDIT_SUM_LIMIT_mean,home_AMT_CREDIT_SUM_OVERDUE_mean,home_AMT_CREDIT_MAX_OVERDUE_mean,home_AMT_ANNUITY_mean,prev_type_loans,prev_cash_loans_perc,prev_revolving_loans_perc,total_accepted_loans
0,100001,3951.000,23787.00,23787.0,24835.50,24835.50,2520.000000,1740.0,8.0,0.0,0.0,0.0,1.0,0.0,5885.132143,5885.132143,9.0,3.0,4.0,0.0,0.0,49.0,0.0,441.571429,-2.0,0.0,1453365.000,207623.571429,596686.5,-2.000000,0.0,-2.000000,3545.357143,1.0,0.0,0.0,8.0
1,100002,9251.775,179055.00,179055.0,179055.00,179055.00,0.000000,606.0,24.0,0.0,0.0,0.0,-1.0,0.0,11559.247105,11559.247105,19.0,2.0,6.0,0.0,0.0,103.0,0.0,115.625000,-2.0,0.0,865055.565,108131.945625,245775.0,3997.570625,0.0,1049.893125,0.000000,1.0,0.0,0.0,9.0
2,100003,56553.990,484191.00,1452573.0,435436.50,435436.50,2294.333333,1305.0,10.0,0.0,0.0,0.0,1.0,1.0,64754.586000,64754.586000,28.0,1.0,3.0,0.0,0.0,606.0,0.0,304.000000,-2.0,0.0,1017400.500,254350.125000,0.0,202500.000000,0.0,0.000000,-2.000000,3.0,33.3,0.0,7.0
3,100004,5357.250,20106.00,20106.0,24282.00,24282.00,4860.000000,815.0,4.0,0.0,0.0,0.0,0.0,0.0,7096.155000,7096.155000,4.0,0.0,2.0,0.0,0.0,408.0,0.0,0.000000,382.0,0.0,189037.800,94518.900000,0.0,0.000000,0.0,-2.000000,0.000000,1.0,0.0,0.0,3.0
4,100005,2406.600,20076.75,40153.5,22308.75,22308.75,2232.000000,536.0,5.0,1.0,0.0,0.0,-1.0,-1.0,6240.205000,6240.205000,11.0,2.0,1.0,0.0,0.0,62.0,0.0,482.000000,-2.0,0.0,657126.000,219042.000000,568408.5,0.000000,0.0,-2.000000,1420.500000,2.0,50.0,0.0,4.0


## 7.3. Les corrélation linéaires

In [48]:
# Matrice des corrélations
corr_mat = final_merge.corr()

# Combinaisons de paires de variables triées par leurs valeurs de corrélation
corr_pairs = corr_mat.unstack()
sorted_pairs = corr_pairs.sort_values(kind="quicksort")

In [49]:
# Les corrélations fortes (> 0.8 ou < -0.8)
strong_pairs = sorted_pairs[(abs(sorted_pairs) >= 0.8) & (abs(sorted_pairs)!=1)]
print(strong_pairs)

prev_AMT_CREDIT_mean       prev_AMT_ANNUITY_mean        0.818680
prev_AMT_ANNUITY_mean      prev_AMT_CREDIT_mean         0.818680
prev_AMT_GOODS_PRICE_mean  prev_AMT_ANNUITY_mean        0.822390
prev_AMT_ANNUITY_mean      prev_AMT_GOODS_PRICE_mean    0.822390
                           prev_AMT_APPLICATION_mean    0.822390
prev_AMT_APPLICATION_mean  prev_AMT_ANNUITY_mean        0.822390
total_accepted_loans       home_closed_sum              0.865356
home_closed_sum            total_accepted_loans         0.865356
prev_AMT_CREDIT_mean       prev_AMT_GOODS_PRICE_mean    0.971721
prev_AMT_GOODS_PRICE_mean  prev_AMT_CREDIT_mean         0.971721
prev_AMT_CREDIT_mean       prev_AMT_APPLICATION_mean    0.971722
prev_AMT_APPLICATION_mean  prev_AMT_CREDIT_mean         0.971722
prev_AMT_PAYMENT_mean      prev_AMT_INSTALMENT_mean     0.979429
prev_AMT_INSTALMENT_mean   prev_AMT_PAYMENT_mean        0.979429
prev_AMT_APPLICATION_mean  prev_AMT_GOODS_PRICE_mean    1.000000
prev_AMT_GOODS_PRICE_mean

**De base, je souhaitais éliminer au moins 2 variables:**
- **La variable AMT_GOODS_PRICE:**
    - Aucun impact pour notre projet
    - Ne concerne que les crédits à la consommation
- **La variable AMT_APPLICATION:**
    - Le principal est la somme réellement octroyée et non celle demandée

**Ces 2 variables montrant de très fortes corrélations avec d'autres variables, mon choix de les éliminer est vraiment justifié.**

**Les autres variables présentant de fortes corrélations seront conservées.**

In [50]:
# Elimination des variables 'prev_AMT_GOODS_PRICE_mean' et 'prev_AMT_APPLICATION_mean'
final_merge = final_merge.drop(["prev_AMT_GOODS_PRICE_mean", "prev_AMT_APPLICATION_mean"], axis=1)

In [51]:
# Informations sur le jeu de données
final_merge.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353577 entries, 0 to 353576
Data columns (total 35 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        353577 non-null  int64  
 1   prev_AMT_ANNUITY_mean             353577 non-null  float64
 2   prev_AMT_CREDIT_mean              353577 non-null  float64
 3   prev_AMT_CREDIT_sum               353577 non-null  float64
 4   prev_AMT_DOWN_PAYMENT_mean        353577 non-null  float64
 5   prev_DAYS_DECISION_mean           353577 non-null  float64
 6   prev_CNT_PAYMENT_mean             353577 non-null  float64
 7   prev_canceled                     353577 non-null  float64
 8   prev_refused                      353577 non-null  float64
 9   prev_unused_offer                 353577 non-null  float64
 10  prev_NAME_TYPE_SUITE              353577 non-null  float64
 11  prev_NFLAG_INSURED_ON_APPROVAL    353577 non-null  f

**ATTENTION**: Présence de valeurs manquantes au niveau des variables 'prev_cash_loans_perc' et 'prev_revolving_loans_perc'. De mon point de vue, il s'agit de clients n'ayant pas d'historique de prêt accepté dans le jeu de données previous_application. Par acquis de conscience, cette hypothèse sera vérifiéé.

In [52]:
unfill_prev_loans = final_merge[(final_merge["prev_cash_loans_perc"].isna()) | (final_merge["prev_revolving_loans_perc"].isna())]
unfill_prev_loans.head(10)

,SK_ID_CURR,prev_AMT_ANNUITY_mean,prev_AMT_CREDIT_mean,prev_AMT_CREDIT_sum,prev_AMT_DOWN_PAYMENT_mean,prev_DAYS_DECISION_mean,prev_CNT_PAYMENT_mean,prev_canceled,prev_refused,prev_unused_offer,prev_NAME_TYPE_SUITE,prev_NFLAG_INSURED_ON_APPROVAL,prev_AMT_PAYMENT_mean,prev_AMT_INSTALMENT_mean,prev_SK_DPD_count,home_active_sum,home_closed_sum,home_bad_debt_sum,home_sold_sum,home_DAYS_CREDIT_min,home_CREDIT_DAY_OVERDUE_mean,home_DAYS_CREDIT_ENDDATE_mean,home_DAYS_ENDDATE_FACT_min,home_CNT_CREDIT_PROLONG_sum,home_AMT_CREDIT_SUM_sum,home_AMT_CREDIT_SUM_mean,home_AMT_CREDIT_SUM_DEBT_sum,home_AMT_CREDIT_SUM_LIMIT_mean,home_AMT_CREDIT_SUM_OVERDUE_mean,home_AMT_CREDIT_MAX_OVERDUE_mean,home_AMT_ANNUITY_mean,prev_type_loans,prev_cash_loans_perc,prev_revolving_loans_perc,total_accepted_loans
338857,100031,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,75.0,0.0,372.571429,-2.0,0.0,3970651.50,5.672359e+05,1.124990e+06,-2.000000,0.0,-2.000000,-2.0,0.0,NaN,NaN,7.0
338858,100058,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,223.0,0.0,7842.750000,-2.0,0.0,810000.00,2.025000e+05,2.585970e+05,-2.000000,0.0,-2.000000,-2.0,0.0,NaN,NaN,4.0
338859,100076,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,121.0,0.0,512.333333,-2.0,0.0,3296057.58,1.098686e+06,2.610860e+06,1582.403333,0.0,16389.413333,61470.0,0.0,NaN,NaN,3.0
338860,100096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2922.0,0.0,0.000000,1647.0,0.0,65610.00,6.561000e+04,-2.000000e+00,-2.000000,0.0,-2.000000,0.0,0.0,NaN,NaN,1.0
338861,100102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,211.0,0.0,641.600000,-2.0,0.0,1473232.50,2.946465e+05,6.943140e+05,0.000000,0.0,-2.000000,-2.0,0.0,NaN,NaN,5.0
338862,100129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,914.0,0.0,6391.000000,-2.0,0.0,10327500.00,1.032750e+07,1.004889e+07,0.000000,0.0,-2.000000,-2.0,0.0,NaN,NaN,1.0
338863,100134,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,271.0,0.0,401.250000,-2.0,0.0,1020408.21,2.551021e+05,2.966990e+03,-2.000000,0.0,-2.000000,-2.0,0.0,NaN,NaN,4.0
338864,100192,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,297.0,0.0,27519.000000,-2.0,0.0,135000.00,1.350000e+05,8.399880e+04,0.000000,0.0,6728.445000,-2.0,0.0,NaN,NaN,1.0
338865,100195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,736.0,0.0,-2.000000,-2.0,0.0,225000.00,1.125000e+05,1.550655e+05,1215.787500,0.0,-2.000000,-2.0,0.0,NaN,NaN,2.0
338866,100203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,532.0,0.0,572.000000,-2.0,0.0,81000.00,8.100000e+04,8.312400e+04,0.000000,0.0,-2.000000,-2.0,0.0,NaN,NaN,1.0


**BINGO**: Les valeurs seront donc imputées par 0.

In [53]:
# Imputation des valeurs par zéro
final_merge = final_merge.fillna(value=0)

In [54]:
# Informations sur le jeu de données
final_merge.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 353577 entries, 0 to 353576
Data columns (total 35 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   SK_ID_CURR                        353577 non-null  int64  
 1   prev_AMT_ANNUITY_mean             353577 non-null  float64
 2   prev_AMT_CREDIT_mean              353577 non-null  float64
 3   prev_AMT_CREDIT_sum               353577 non-null  float64
 4   prev_AMT_DOWN_PAYMENT_mean        353577 non-null  float64
 5   prev_DAYS_DECISION_mean           353577 non-null  float64
 6   prev_CNT_PAYMENT_mean             353577 non-null  float64
 7   prev_canceled                     353577 non-null  float64
 8   prev_refused                      353577 non-null  float64
 9   prev_unused_offer                 353577 non-null  float64
 10  prev_NAME_TYPE_SUITE              353577 non-null  float64
 11  prev_NFLAG_INSURED_ON_APPROVAL    353577 non-null  f

**Nous sommes désormais en présence d'un jeu de données complet pouvant être sauvegardé pour la jointure avec les jeux d'entraînement et de test dans le prochain notebook.**

In [55]:
# Sauvegarde du jeu de données
complete_history = final_merge.to_csv("complete_history.csv", index=False)

### Conclusion de ce notebook

**Etant dans une optique d'apprentissage et ce domaine m'intéressant (bien que biologiste de base), en accord avec mon mentor travaillant dans ce domaine, le fait de ne pas être partie du kernel de Kaggle m'a pris énormément de temps MAIS a été très enrichissant.**

**A NOTER:** Ce notebook sera nommé 'Ple_Coline_3_notebook_historique_complet_092023' dans les livrables et 'historique_complet' sur GitHub.